# Extract and align data from Onix, Harp, Sleap, and photometry

In [ ]:
import numpy as np
from pathlib import Path
import os
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import pandas as pd
import harp

from harp_resources import process, utils
from sleap import load_and_process as lp

In [ ]:
cohort0 = False #only read harp data when it exists, not in Cohort0 

#Cohort 1 single OnixDigital file
#data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort1/VestibularMismatch_day1/B6J2717-2024-12-12T13-00-21') #single onix_digital files

#Cohort 1 multiple OnixDigital files 
#data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort1/VestibularMismatch_day1/B6J2718-2024-12-12T13-28-14') #multiple onix_digital file

#Cohort 0 (no OnixHarp in this Cohort)
#data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort0/Cohort0_GCaMP_example/B3M3xx-2024-08-08T10-05-26')
#cohort0 = True

#Cohort 2 N.B. no videodata in this test set 
data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort2_like_test_data/2025-01-13T15-47-26')

photometry_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort1/VestibularMismatch_day1/B6J2717-2024-12-12T13-00-21_processedData/photometry')

h1_datafolder = data_path / 'HarpDataH1' #only if reading separate registers
h2_datafolder = data_path / 'HarpDataH2' #only if reading separate registers

In [ ]:
#h1 and h2 only needed if timestamps are readed separately and not as all harp_streams
h1_reader = harp.create_reader('harp_resources/h1-device.yml', epoch=harp.REFERENCE_EPOCH)
h2_reader = harp.create_reader('harp_resources/h2-device.yml', epoch=harp.REFERENCE_EPOCH)
session_settings_reader = utils.SessionData("SessionSettings")
experiment_events_reader = utils.TimestampedCsvReader("ExperimentEvents", columns=["Event"])
onix_framecount_reader = utils.TimestampedCsvReader("OnixAnalogFrameCount", columns=["Index"])
photometry_reader = utils.PhotometryReader("Processed_fluorescence")
video_reader1 = utils.Video("VideoData1")
video_reader2 = utils.Video("VideoData2")
onix_digital_reader = utils.OnixDigitalReader("OnixDigital", columns=["Value.Clock", "Value.HubClock", 
                                                                         "Value.DigitalInputs",
                                                                         "Seconds"])
onix_harp_reader = utils.TimestampedCsvReader("OnixHarp", columns=["Clock", "HubClock", "HarpTime"])

In [ ]:
import importlib
importlib.reload(utils)  # Forces Python to reload the updated module
if not cohort0:
    print ("Loading OnixHarp")
    onix_harp = utils.load_2(onix_harp_reader, data_path)

print ("Loading H1 and H2 streams as dict or df")
harp_streams = utils.load_registers(data_path, dataframe = True, verbose = False) #loads as df, or if False, as dict 

#print ("Loading OnixAnalogFrameClock")
#onix_analog_framecount = utils.load_2(onix_framecount_reader, data_path)
print ("Done Loading")

In [ ]:
harp_streams

In [ ]:
onix_harp

In [ ]:
import pandas as pd

df = pd.DataFrame(experiment_events
                 )
column_name = "Seconds"

first_index = df.index[0]
last_index = df.index[-1]
time_difference = (last_index - first_index).total_seconds()

first_index, first_value = df.index[0], df[column_name].iloc[0]
last_index, last_value = df.index[-1], df[column_name].iloc[-1]
difference = last_value - first_value

# Print the result
print(f"Index Difference (seconds): {time_difference}")
print(f"Difference in Seconds column: {difference}")


In [ ]:
print (90586.99024-88999.99024)
print (69*60+46.990240 - 43*60+19.990240)

In [ ]:
video_data

In [ ]:
#read metadata in 2 different ways (to df or to dict, to decide which one is better in the future)
print ("Loading session settings")
session_settings = utils.load_2(session_settings_reader, data_path) #Andrew's, creates ugly df, but used in further analysis code
#session_settings = utils.read_SessionSettings(data_path) #Hilde's, creates prety dict, not aware of multiple files

# read experiment events, video, processed photometry 
print ("Loading experiment events")
experiment_events = utils.load_2(experiment_events_reader, data_path)
print ("Loading processed fluorescence")
photometry_data=pd.read_csv(str(photometry_path)+'/Processed_fluorescence.csv')
print ("Loading processed fluorescence info")
photometry_info=pd.read_csv(str(photometry_path)+'/Info.csv')
print ("Loading processed fluorescence events")
photometry_events=pd.read_csv(str(photometry_path)+'/Events.csv')
print ("Loading video data 1")
video_data1 = utils.load_2(video_reader1, data_path)
print ("Loading video data 2")
video_data2 = utils.load_2(video_reader2, data_path)

# read Onix data 
print ("Loading OnixDigital")
onix_digital = utils.load_2(onix_digital_reader, data_path)
if not cohort0:
    print ("Loading OnixHarp")
    onix_harp = utils.load_2(onix_harp_reader, data_path)
print ("Loading OnixAnalogFrameClock")
onix_analog_framecount = utils.load_2(onix_framecount_reader, data_path)
print ("Loading OnixAnalogClock")
onix_analog_clock = utils.read_OnixAnalogClock(data_path)
print ("Loading OnixAnalogData")
onix_analog_data = utils.read_OnixAnalogData(data_path, channels = [0], binarise=True) #channels is a list of AI lines, 0-11

#read harp streams and separate registers if needed 
print ("Loading H1 and H2 streams as dict or df")
harp_streams = utils.load_registers(data_path, dataframe = True) #loads as df, or if False, as dict 
print (" ")
print ("loading separate registers from H1 and H2 data")
print ("Loading camera triggers")
camera_triggers = utils.load_harp(h1_reader.Cam0Event, h1_datafolder) #assumes Cam0 triggers both cameras
print ("Loading flow sensor data")
flow_sensor = utils.load_harp(h1_reader.OpticalTrackingRead, h1_datafolder)
print ("Done Loading")

In [ ]:
%whos

### DEV align Onix, HARP and Photometry data 

In [ ]:
import importlib
importlib.reload(utils)  # Forces Python to reload the updated module
None

In [ ]:
onix_harp

In [ ]:
onix_digital

In [ ]:
import plotly.express as px

# Ensure `Time` is not both an index and a column
if "Time" in onix_harp.columns:
    onix_harp = onix_harp.reset_index(drop=True)
else:
    onix_harp = onix_harp.reset_index()

if "Time" in onix_digital.columns:
    onix_digital = onix_digital.reset_index(drop=True)
else:
    onix_digital = onix_digital.reset_index()

# Merge the DataFrames on the `Time` column
merged_data = pd.merge(onix_harp, onix_digital, on="Time", suffixes=("_harp", "_digital"))

# Create the scatter plot
fig = px.scatter(
    merged_data,
    x="Clock_digital",  # X-axis: Clock from onix_digital
    y="HarpTime",       # Y-axis: HarpTime from onix_harp
    title="HarpTime vs Clock",
    labels={"Clock_digital": "Clock (onix_digital)", "HarpTime": "HarpTime (onix_harp)"}
)

# Add interactivity (zoom, pan)
fig.update_layout(
    xaxis_title="Clock (onix_digital)",
    yaxis_title="HarpTime (onix_harp)",
    dragmode="pan"  # Enable panning by default
)

# Show the plot
fig.show()

